<a href="https://colab.research.google.com/github/athira-tech1/C-_-/blob/main/Existingproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python cryptography




In [ ]:
import cv2
import hashlib
import json
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey, Ed25519PublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature
import os
from google.colab import files

# Function to extract frames and hash them
def extract_and_hash(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_hashes = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Convert frame to bytes and compute SHA-256 hash
        frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
        frame_hash = hashlib.sha256(frame_bytes).hexdigest()
        frame_hashes.append(frame_hash)
        frame_count += 1

    cap.release()
    return frame_hashes

# Function to sign the hashes
def sign_hashes(private_key, hashes):
    # Combine all hashes into one string and sign it
    combined_hash = ''.join(hashes).encode()
    signature = private_key.sign(combined_hash)
    return signature

# Function to verify integrity
def verify_hashes(public_key, original_hashes, altered_hashes, signature):
    # Combine original hashes and verify signature
    combined_original_hash = ''.join(original_hashes).encode()
    try:
        public_key.verify(signature, combined_original_hash)
    except InvalidSignature:
        return False, "Invalid signature. Video integrity cannot be verified."

    # Compare frame-wise hashes
    if original_hashes == altered_hashes:
        return True, "Video is authentic and untampered."
    else:
        return False, "Video has been tampered with!"

# Save keys to files
def save_keys(private_key, public_key):
    private_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    with open("private_key.pem", "wb") as f:
        f.write(private_bytes)
    with open("public_key.pem", "wb") as f:
        f.write(public_bytes)

# Load keys from files
def load_keys():
    with open("private_key.pem", "rb") as f:
        private_key = serialization.load_pem_private_key(f.read(), password=None)
    with open("public_key.pem", "rb") as f:
        public_key = serialization.load_pem_public_key(f.read())
    return private_key, public_key

# Function to handle video upload
def upload_video(prompt):
    print(prompt)
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]
    return video_path

# Main process
if __name__ == "__main__":
    # Generate keys if not present
    if not os.path.exists("private_key.pem") or not os.path.exists("public_key.pem"):
        private_key = Ed25519PrivateKey.generate()
        public_key = private_key.public_key()
        save_keys(private_key, public_key)
    else:
        private_key, public_key = load_keys()

    # Upload original video
    original_video_path = upload_video("Upload the original video:")
    original_hashes = extract_and_hash(original_video_path)

    # Sign and store hashes
    signature = sign_hashes(private_key, original_hashes)
    with open("hashes.json", "w") as f:
        json.dump({"hashes": original_hashes, "signature": signature.hex()}, f)

    print("Original video hashes and signature stored successfully.")

    # Upload altered video
    altered_video_path = upload_video("Upload the altered video:")
    altered_hashes = extract_and_hash(altered_video_path)

    # Verify integrity
    with open("hashes.json", "r") as f:
        data = json.load(f)
        stored_hashes = data["hashes"]
        stored_signature = bytes.fromhex(data["signature"])

    is_valid, message = verify_hashes(public_key, stored_hashes, altered_hashes, stored_signature)
    print(message)


Upload the original video:


Saving SampleVideo_1280x720_2mb.mp4 to SampleVideo_1280x720_2mb.mp4
Original video hashes and signature stored successfully.
Upload the altered video:
